In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import os
from torch.nn.utils.rnn import pad_sequence
notebook_path = os.path.abspath('')
import sys
# Find the part of the path that contains 'commitFit'
commit_fit_path = None
for part in notebook_path.split(os.sep):
    print(part)
    if 'ensemble_commit' in part:
        commit_fit_path = notebook_path.split(part)[0] + part
        print(commit_fit_path)
        break

# import whatthepatch
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_curve,classification_report
from tqdm import tqdm as std_tqdm
from functools import partial
tqdm = partial(std_tqdm, dynamic_ncols=True)


root
ensemble_commit
/root/ensemble_commit


In [2]:
df = pd.read_json(r'../datasets/patch_db.json', encoding='utf_8_sig')
df.dropna(inplace=True)
label2id={'non-security':0,'security':1}
df = df.replace({"category": label2id})
df

/tmp/ipykernel_2333/183603274.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"category": label2id})


,CVE_ID,CWE_ID,category,commit_id,commit_message,diff_code,owner,repo,source
0,NA,NA,0,540958e2f5a87b81aa5f55ce40b3e2869754f97d,commit 540958e2f5a87b81aa5f55ce40b3e2869754f97...,diff --git a/drivers/staging/comedi/drivers/cb...,stoth68000,media-tree,wild
1,NA,NA,0,64d240b721b21e266ffde645ec965c3b6d1c551f,commit 64d240b721b21e266ffde645ec965c3b6d1c551...,diff --git a/drivers/target/target_core_file.c...,stoth68000,media-tree,wild
2,NA,NA,0,f181dd278274f50e689ebd13237010a90b430164,commit f181dd278274f50e689ebd13237010a90b43016...,diff --git a/include/paths.h b/include/paths.h...,openbsd,src,wild
3,NA,NA,0,0abdc3723b5d33dde698ab941325edec2819c128,commit 0abdc3723b5d33dde698ab941325edec2819c12...,diff --git a/gnu/usr.bin/binutils/ld/lexsup.c ...,openbsd,src,wild
4,NA,NA,0,d7930d7f820e5dd6b07b823f155aeb943b525e16,commit d7930d7f820e5dd6b07b823f155aeb943b525e1...,diff --git a/src/expat_erl.c b/src/expat_erl.c...,esl,MongooseIM,wild
...,...,...,...,...,...,...,...,...,...
35810,CVE-2013-0217,399,1,7d5145d8eb2b9791533ffe4dc003b129b9696c48,From 7d5145d8eb2b9791533ffe4dc003b129b9696c48 ...,diff --git a/drivers/net/xen-netback/netback.c...,torvalds,linux,cve
35811,CVE-2018-18311,119,1,34716e2a6ee2af96078d62b065b7785c001194be,From 34716e2a6ee2af96078d62b065b7785c001194be ...,diff --git a/util.c b/util.c\nindex 7282dd9cfe...,Perl,perl5,cve
35812,CVE-2019-12984,476,1,385097a3675749cbc9e97c085c0e5dfe4269ca51,From 385097a3675749cbc9e97c085c0e5dfe4269ca51 ...,diff --git a/net/nfc/netlink.c b/net/nfc/netli...,torvalds,linux,cve
35813,CVE-2013-0865,119,1,f3d16706060ab6ae6dc78f15359fab3fd87c9495,From f3d16706060ab6ae6dc78f15359fab3fd87c9495 ...,diff --git a/libavcodec/vqavideo.c b/libavcode...,NA,NA,cve


In [3]:
train_data, test_data = train_test_split(df, test_size=0.3, random_state=42)
test_data, val_data = train_test_split(test_data, test_size=0.5, random_state=42)
# train_data = pd.read_csv('./datasets/PD_train.csv')
# test_data = pd.read_csv('./datasets/PD_test.csv')
# val_data = pd.read_csv('./datasets/PD_val.csv')



In [4]:
# codebert_model.config.hidden_size

In [5]:
len(test_data)

5372

In [6]:
train_data,_ = train_test_split(train_data, train_size=0.1, random_state=42)
test_data, val_data = train_test_split(test_data, train_size=0.1, random_state=42)
val_data, _ = train_test_split(val_data, train_size=0.1, random_state=42)

In [7]:
train_data.reset_index(inplace=True)
test_data.reset_index(inplace=True)
val_data.reset_index(inplace=True)

In [8]:
train_data['category'].value_counts()

category
0    1644
1     863
Name: count, dtype: int64

In [9]:
val_data

,index,CVE_ID,CWE_ID,category,commit_id,commit_message,diff_code,owner,repo,source
0,26915,NA,NA,1,44241628bb207ec211bebd156aaf69470d90c209,commit 44241628bb207ec211bebd156aaf69470d90c20...,diff --git a/drivers/thermal/power_allocator.c...,stoth68000,media-tree,wild
1,18930,NA,NA,0,fe82772c377e1c0bdb1ce724f8bbe8ceb6294ee1,commit fe82772c377e1c0bdb1ce724f8bbe8ceb6294ee...,diff --git a/core/fork_server.c b/core/fork_se...,unbit,uwsgi,wild
2,15526,NA,NA,0,625c6ace93bb1515ff16840741d6f7567be6ebb8,commit 625c6ace93bb1515ff16840741d6f7567be6ebb...,diff --git a/send.c b/send.c\nindex 2ced24b..9...,reubenhwk,radvd,wild
3,13501,NA,NA,0,8a7cf3febbb2b7c1ade717ddb3065de67c5983c5,commit 8a7cf3febbb2b7c1ade717ddb3065de67c5983c...,diff --git a/drivers/net/wireless/wl12xx/main....,mjg59,linux,wild
4,10592,NA,NA,0,5464baecf521d1ca8095604f5a7371443c94226a,commit 5464baecf521d1ca8095604f5a7371443c94226...,diff --git a/linux-user/syscall.c b/linux-user...,qemu,qemu,wild
...,...,...,...,...,...,...,...,...,...,...
478,12654,NA,NA,0,056790923e1c4eed5d8cc502e1092944a2b23025,commit 056790923e1c4eed5d8cc502e1092944a2b2302...,diff --git a/sound/soc/soc-pcm.c b/sound/soc/s...,stoth68000,media-tree,wild
479,7284,NA,NA,0,53155c0a5949d5cc3bd434d838e5b0c6e0542f9b,commit 53155c0a5949d5cc3bd434d838e5b0c6e0542f9...,diff --git a/drivers/gpu/drm/i915/intel_displa...,torvalds,linux,wild
480,22406,NA,NA,0,7b96dc023a1b487bce59256fde14b8bb28b45aea,commit 7b96dc023a1b487bce59256fde14b8bb28b45ae...,diff --git a/drivers/sbus/char/vfc.h b/drivers...,torvalds,linux,wild
481,26222,NA,NA,1,aa9a4a1edfbd3d223af01db833da2f07850bc655,commit aa9a4a1edfbd3d223af01db833da2f07850bc65...,diff --git a/fs/pstore/ram.c b/fs/pstore/ram.c...,stoth68000,media-tree,wild


In [10]:
val_data

,index,CVE_ID,CWE_ID,category,commit_id,commit_message,diff_code,owner,repo,source
0,26915,NA,NA,1,44241628bb207ec211bebd156aaf69470d90c209,commit 44241628bb207ec211bebd156aaf69470d90c20...,diff --git a/drivers/thermal/power_allocator.c...,stoth68000,media-tree,wild
1,18930,NA,NA,0,fe82772c377e1c0bdb1ce724f8bbe8ceb6294ee1,commit fe82772c377e1c0bdb1ce724f8bbe8ceb6294ee...,diff --git a/core/fork_server.c b/core/fork_se...,unbit,uwsgi,wild
2,15526,NA,NA,0,625c6ace93bb1515ff16840741d6f7567be6ebb8,commit 625c6ace93bb1515ff16840741d6f7567be6ebb...,diff --git a/send.c b/send.c\nindex 2ced24b..9...,reubenhwk,radvd,wild
3,13501,NA,NA,0,8a7cf3febbb2b7c1ade717ddb3065de67c5983c5,commit 8a7cf3febbb2b7c1ade717ddb3065de67c5983c...,diff --git a/drivers/net/wireless/wl12xx/main....,mjg59,linux,wild
4,10592,NA,NA,0,5464baecf521d1ca8095604f5a7371443c94226a,commit 5464baecf521d1ca8095604f5a7371443c94226...,diff --git a/linux-user/syscall.c b/linux-user...,qemu,qemu,wild
...,...,...,...,...,...,...,...,...,...,...
478,12654,NA,NA,0,056790923e1c4eed5d8cc502e1092944a2b23025,commit 056790923e1c4eed5d8cc502e1092944a2b2302...,diff --git a/sound/soc/soc-pcm.c b/sound/soc/s...,stoth68000,media-tree,wild
479,7284,NA,NA,0,53155c0a5949d5cc3bd434d838e5b0c6e0542f9b,commit 53155c0a5949d5cc3bd434d838e5b0c6e0542f9...,diff --git a/drivers/gpu/drm/i915/intel_displa...,torvalds,linux,wild
480,22406,NA,NA,0,7b96dc023a1b487bce59256fde14b8bb28b45aea,commit 7b96dc023a1b487bce59256fde14b8bb28b45ae...,diff --git a/drivers/sbus/char/vfc.h b/drivers...,torvalds,linux,wild
481,26222,NA,NA,1,aa9a4a1edfbd3d223af01db833da2f07850bc655,commit aa9a4a1edfbd3d223af01db833da2f07850bc65...,diff --git a/fs/pstore/ram.c b/fs/pstore/ram.c...,stoth68000,media-tree,wild


In [11]:
import pandas as pd
import gensim
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np

# 构建自定义数据集
class TextDataset(Dataset):
    def __init__(self, df, message_vocab, code_vocab, max_len):
        self.messages = df["commit_message"].apply(lambda x: self.encode_text(x.split(), message_vocab, max_len)).tolist()
        self.codes = df["diff_code"].apply(lambda x: self.encode_text(x.split(), code_vocab, max_len)).tolist()
        self.labels = df["category"].tolist()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        messages = np.array(self.messages[idx], dtype=np.int64)
        codes = np.array(self.codes[idx], dtype=np.int64)
        label = int(self.labels[idx])  # 确保单个标量是整数类型
        return messages, codes, label

    @staticmethod
    def encode_text(text, vocab, max_len):
        encoded = [vocab[word] if word in vocab else 0 for word in text]
        return encoded + [0] * (max_len - len(encoded))


def collate_fn(batch):
    messages, codes, labels = zip(*batch)
    
    # 确保 messages 和 codes 是二维张量，并填充
    messages = pad_sequence([torch.tensor(msg, dtype=torch.long) for msg in messages], batch_first=True, padding_value=0)
    codes = pad_sequence([torch.tensor(code, dtype=torch.long) for code in codes], batch_first=True, padding_value=0)
    
    # 确保 labels 是一维张量
    labels = torch.tensor(labels, dtype=torch.long)
    return messages, codes, labels
    
# 使用 gensim 训练 Word2Vec 模型
def train_word2vec(sentences, vector_size=100, window=5, min_count=1):
    model = gensim.models.Word2Vec(sentences, vector_size=vector_size, window=window, min_count=min_count)
    return model

# 定义 LSTM + CNN 模型
class LSTM_CNN_Model(nn.Module):
    def __init__(self, message_vocab_size, code_vocab_size, embedding_dim, hidden_dim, lstm_layers, cnn_out_channels, num_classes, 
                 message_embedding_matrix=None, code_embedding_matrix=None):
        super(LSTM_CNN_Model, self).__init__()
        
        # Embedding Layers for message and code
        self.message_embedding = nn.Embedding(message_vocab_size, embedding_dim)
        self.code_embedding = nn.Embedding(code_vocab_size, embedding_dim)
        
        if message_embedding_matrix is not None:
            self.message_embedding.weight = nn.Parameter(torch.tensor(message_embedding_matrix, dtype=torch.float32))
            self.message_embedding.weight.requires_grad = False
        
        if code_embedding_matrix is not None:
            self.code_embedding.weight = nn.Parameter(torch.tensor(code_embedding_matrix, dtype=torch.float32))
            self.code_embedding.weight.requires_grad = False
        
        # LSTM Layers
        self.message_lstm = nn.LSTM(embedding_dim, hidden_dim, lstm_layers, batch_first=True, bidirectional=True)
        self.code_lstm = nn.LSTM(embedding_dim, hidden_dim, lstm_layers, batch_first=True, bidirectional=True)
        
        # CNN Layers
        self.message_conv1d = nn.Conv1d(in_channels=hidden_dim * 2, out_channels=cnn_out_channels, kernel_size=3, stride=1, padding=1)
        self.code_conv1d = nn.Conv1d(in_channels=hidden_dim * 2, out_channels=cnn_out_channels, kernel_size=3, stride=1, padding=1)
        
        # Fully Connected Layer
        self.fc = nn.Linear(cnn_out_channels * 2, num_classes)

    def forward(self, message, code):
        # Process message
        message_embed = self.message_embedding(message)
        message_lstm_out, _ = self.message_lstm(message_embed)
        message_lstm_out = message_lstm_out.permute(0, 2, 1)  # Change to (batch, channels, seq_len)
        message_features = torch.relu(self.message_conv1d(message_lstm_out))
        message_features = torch.mean(message_features, dim=2)  # Global Average Pooling
        
        # Process code
        code_embed = self.code_embedding(code)
        code_lstm_out, _ = self.code_lstm(code_embed)
        code_lstm_out = code_lstm_out.permute(0, 2, 1)  # Change to (batch, channels, seq_len)
        code_features = torch.relu(self.code_conv1d(code_lstm_out))
        code_features = torch.mean(code_features, dim=2)  # Global Average Pooling
        
        # Combine features
        combined_features = torch.cat([message_features, code_features], dim=1)
        output = self.fc(combined_features)
        return output



In [12]:
len(train_data)

2507

In [13]:
# 分别训练 Word2Vec 模型
message_sentences = [msg.split() for msg in train_data["commit_message"]]
code_sentences = [code.split() for code in train_data["diff_code"]]
message_word2vec = train_word2vec(message_sentences, vector_size=100)
code_word2vec = train_word2vec(code_sentences, vector_size=100)

# 构建词汇表
message_vocab = {word: idx for idx, word in enumerate(message_word2vec.wv.index_to_key)}
code_vocab = {word: idx for idx, word in enumerate(code_word2vec.wv.index_to_key)}

# 构建嵌入矩阵
message_embedding_matrix = np.zeros((len(message_vocab), 100))
for word, idx in message_vocab.items():
    message_embedding_matrix[idx] = message_word2vec.wv[word]

code_embedding_matrix = np.zeros((len(code_vocab), 100))
for word, idx in code_vocab.items():
    code_embedding_matrix[idx] = code_word2vec.wv[word]

# 数据处理
max_len = 10
train_dataset = TextDataset(train_data, message_vocab, code_vocab, max_len)
# 创建 DataLoader 时指定 collate_fn
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)

In [14]:
# device = torch.device('cpu')

In [15]:
test_dataset = TextDataset(test_data, message_vocab, code_vocab, max_len)
# 创建 DataLoader 时指定 collate_fn
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)

In [16]:
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate(model, data_loader, criterion):
    """
    Evaluate the model on the test dataset, computing loss, accuracy, precision, recall, and F1 score.
    
    Args:
        model: PyTorch model to evaluate.
        data_loader: DataLoader for the test dataset.
        criterion: Loss function to evaluate the model's performance.
        device: Device to run the evaluation (e.g., "cpu" or "cuda").
    
    Returns:
        avg_loss: Average loss on the test dataset.
        accuracy: Accuracy of the model on the test dataset.
        precision: Precision score for the test dataset.
        recall: Recall score for the test dataset.
        f1: F1 score for the test dataset.
    """
    model.eval()  # Set the model to evaluation mode
    total_loss = 0.0
    total_correct = 0
    total_samples = 0
    all_labels = []
    all_preds = []

    with torch.no_grad():  # Disable gradient computation for evaluation
        for messages, codes, labels in data_loader:
            # Move data to the specified device
            messages, codes, labels = torch.tensor(messages), torch.tensor(codes), torch.tensor(labels)
            
            # Forward pass
            outputs = model(messages, codes)
            loss = criterion(outputs, labels)
            
            # Update loss
            total_loss += loss.item() * labels.size(0)  # Multiply by batch size
            
            # Compute predictions
            _, predicted = torch.max(outputs, dim=1)
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)
            
            # Collect predictions and true labels for metrics
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())
            
    # Compute average loss and accuracy
    avg_loss = total_loss / total_samples
    accuracy = total_correct / total_samples
    
    # Compute precision, recall, and F1 score
    precision = precision_score(all_labels, all_preds, average="weighted", zero_division=0)
    recall = recall_score(all_labels, all_preds, average="weighted", zero_division=0)
    f1 = f1_score(all_labels, all_preds, average="weighted", zero_division=0)
    print(f'Validation Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1-Score: {f1}')
    
    return all_labels, all_preds


In [18]:
model = torch.load("SPI_entire_bert_model.pth")

In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [20]:
# Evaluate the model
test_labels, test_predictions = evaluate(model, test_loader, criterion)

/tmp/ipykernel_2333/2610449562.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  messages, codes, labels = torch.tensor(messages), torch.tensor(codes), torch.tensor(labels)


Validation Accuracy: 0.7877094972067039
Precision: 0.832813589896508
Recall: 0.7877094972067039
F1-Score: 0.7481121061261119


In [21]:
print(classification_report(test_labels,test_predictions,digits=4))

              precision    recall  f1-score   support

           0     0.7656    0.9973    0.8662       370
           1     0.9818    0.3234    0.4865       167

    accuracy                         0.7877       537
   macro avg     0.8737    0.6603    0.6763       537
weighted avg     0.8328    0.7877    0.7481       537



In [ ]:
#发送多种类型的邮件
from email.mime.multipart import MIMEMultipart
import smtplib

from email.mime.text import MIMEText
msg_from = '915803745@qq.com'  # 发送方邮箱
passwd = 'vcuosuurrgkfbdai'   #就是上面的授权码
 
# to= ['g.zhang@gotion.com', 'j.tong@gotion.com'] #接受方邮箱
to= ['j.tong@gotion.com'] #接受方邮箱
#设置邮件内容
#MIMEMultipart类可以放任何内容
msg = MIMEMultipart()
conntent="在线模型训练完毕"
#把内容加进去
msg.attach(MIMEText(conntent,'plain','utf-8'))
 
#设置邮件主题
msg['Subject']="在线模型训练完毕"
 
#发送方信息
msg['From']=msg_from
 
#开始发送
 
#通过SSL方式发送，服务器地址和端口
s = smtplib.SMTP_SSL("smtp.qq.com", 465)
# 登录邮箱
s.login(msg_from, passwd)
#开始发送
s.sendmail(msg_from,to,msg.as_string())
print("在线模型训练完毕")